# Get train and validation datasets to local test
Из-за ограничения в 2 загрузки в день решения на kaggle сначала гоняю алгоритмы на собственных тестах


In [19]:
import pandas as pd
import numpy as np

source_dataset_path = './train/train_dataset.csv'
train_path = './fit/train.csv'
validation_path = './fit/validation.csv'

train_df = pd.read_csv(source_dataset_path, sep=',')

Получаем выборки: тусуем, делим в соотношении 90/10 - данных слишком мало

In [20]:
ratio = 0.9

def get_train_and_validation_datasets(train_df):
    indexes = np.arange(0, train_df.shape[0])
    np.random.shuffle(indexes)
    pos = int(train_df.shape[0] * ratio)
    train_indexes = indexes[:pos]
    validation_indexes = indexes[pos:]
    train = train_df.loc[train_indexes]
    validation = train_df.loc[validation_indexes]
    return train, validation

train, validation = get_train_and_validation_datasets(train_df)
train.to_csv(train_path, sep=',')
validation.to_csv(validation_path, sep=',')